In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this seems to be highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown
# note that GradientTape needs several times the memory needed to compute the fidelity of a single circuit
import tensorflow as tf
import numpy as np
import qutip as qt
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import GRAPE
import matplotlib.pyplot as plt


Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0



In [ ]:
q_dim = 4
c_dim = 10
DAC_time_resolution = 2 # in ns
fock = 1

# In GHz = cycles / ns
anharm = -.180
kerr = -1e-5
chi = -4e-3
drive = D = 2 * np.pi * 1e-3

a = qt.tensor(qt.destroy(c_dim), qt.qeye(q_dim))
b = qt.tensor(qt.qeye(c_dim), qt.destroy(q_dim))
gf = qt.tensor(qt.qeye(c_dim), qt.basis(q_dim, 0) * qt.basis(q_dim, 2).dag())
fg = gf.dag()
ad = a.dag()
bd = b.dag()
H0 = (anharm/2) * bd * bd * b * b
H0 += (kerr/2) * ad * ad * a * a
H0 += (chi) * ad * a * bd * b
H0 *= 2*np.pi
Hcs = [D*(gf + fg), 1j*D*(gf - fg), D*(a + ad), 1j*D*(a - ad)]

init_states = [
    qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 0))
]

final_states = [
    qt.tensor(qt.basis(c_dim, fock), qt.basis(q_dim, 0))
]

In [2]:
q_dim = 4
c_dim = 10
DAC_time_resolution = 1
fock = 1

# In GHz = cycles / ns
anharm = -.180
kerr = -1e-5
chi = -4e-3
drive = D = 2 * np.pi * 1e-3

a = qt.tensor(qt.destroy(c_dim), qt.qeye(q_dim))
b = qt.tensor(qt.qeye(c_dim), qt.destroy(q_dim))
# gf = qt.tensor(qt.qeye(c_dim), qt.basis(q_dim, 0) * qt.basis(q_dim, 2).dag())
# fg = gf.dag()
ad = a.dag()
bd = b.dag()
H0 = (anharm/2) * bd * bd * b * b
H0 += (kerr/2) * ad * ad * a * a
H0 += (chi) * ad * a * bd * b
H0 *= 2*np.pi
Hcs = [D*(b + bd), 1j*D*(b - bd), D*(a + ad), 1j*D*(a - ad)] # I, Q, I, Q

init_states = [
    qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 0))
]

final_states = [
    qt.tensor(qt.basis(c_dim, fock), qt.basis(q_dim, 0))
]

In [3]:
synth_params = {
    'N_blocks': 251, # note that the length of the pulse is this times the DAC_time_resolution
    'N_multistart' : 15, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 300, #number of epochs before termination
    'epoch_size' : 5, #number of adam steps per epoch
    'learning_rate' : 0.05, #adam learning rate
    'term_fid' : 0.995, #0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'initial_states' : init_states, #qubit tensor oscillator, start in |g> |0>
    'target_states' : final_states, #end in |e> |target>.
    'name' : 'GRAPE Fock %d' % fock, #name for printing and saving
    'use_phase' : True,
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}

# We initialize the gateset here
gate_set_params = {
    'H_static' : H0,
    'H_control' : Hcs,
    'DAC_delta_t' : DAC_time_resolution,
    'inplace' : False, # true uses less memory, but is slower. Just use false
    'scale' : 1.0, # range of DAC amplitudes for initial random waves
    'bandwidth' : 0.1, #0.05
    'gatesynthargs': synth_params
}



GRAPE_gate_set = GRAPE(**gate_set_params)

2022-06-09 15:29:05.148651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 15:29:05.176266: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 15:29:05.176460: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-09 15:29:05.176851: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate 

In [4]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(GRAPE_gate_set)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
GRAPE_gate_set.best_fidelity()

2022-06-09 15:29:06.831909: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x7fcd3c0084c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-06-09 15:29:06.831946: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2022-06-09 15:29:06.843823: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:237] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-06-09 15:29:07.926763: I tensorflow/compiler/jit/xla_compilation_cache.cc:402] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2022-06-09 15:29:08.052180: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x7088610


6.473418e-05

In [5]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
# Note that you will not want to use the performance profiler while using 'inplace' mode. You will run out of memory
opt.optimize()#logdir='logs')

Start time: 2022-06-09 15:29:08
 Epoch: 293 / 300 Max Fid: 0.995073 Avg Fid: 0.994554 Max dFid: 0.000198 Avg dFid: 0.000184 Elapsed time: 0:06:59.553071 Expected remaining time: 0:00:10.023452

 Optimization stopped. Term fidelity reached.

name: GRAPE Fock 1
scale: 1.0
N_blocks: 251
optimization_type: state transfer
optimization_masks: {'I_DC0': None, 'I_real0': None, 'I_imag0': None, 'Q_DC0': None, 'Q_real0': None, 'Q_imag0': None, 'I_DC1': None, 'I_real1': None, 'I_imag1': None, 'Q_DC1': None, 'Q_real1': None, 'Q_imag1': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 15
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.05
epoch_size: 5
epochs: 300
filename: None
comment: 
use_phase: True
timestamps: ['2022-06-09 15:29:08']
do_prints: True
filename: GRAPE Fock 1.h5

Best circuit parameters found:
I0:    tf.Tensor(
[-3.29968 -3.0901  -2.75283 -2.32553 -1.84976 -1.36769 -0.91888 -0.53726
 -0.2487  -0.06928 -0.00432 -0.04834 -0.18586

'2022-06-09 15:29:08'

In [ ]:
GRAPE_gate_set.print_info()

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs

In [ ]:
GRAPE.get_IQ_time_series

In [ ]:
import h5py
f = h5py.File('/workspaces/QOGS/examples/GRAPE Fock 1.h5', 'r')
pulse_obj = f[list(f.keys())[len(list(f.keys())) - 1]] # get the latest key
# pulse_obj = f['2022-06-08 18:42:44']
list(f.keys())

In [ ]:
fids = pulse_obj['fidelities']
fids.shape
pulse_idx = np.argmax(np.amax(fids, axis=0))

In [ ]:
test = GRAPE_gate_set.get_IQ_time_series(pulse_obj['I_DC1'][-1, :,:].T, pulse_obj['I_real1'][-1, :,:].T,
                            pulse_obj['I_imag1'][-1, :,:].T,
                            pulse_obj['Q_DC1'][-1, :,:], pulse_obj['Q_real1'][-1, :,:].T,
                            pulse_obj['Q_imag1'][-1, :,:].T)

In [ ]:
zeros = np.zeros((301), dtype=np.complex64)
zeros[1] = (1-1j)
zeros[-1] = (1+1j)
zeros[10] = (1+1j) * 1j
zeros[-10] = (1-1j) * 1j
boring = tf.constant(zeros, dtype=tf.complex64)

# plt.plot(test[0:50, :])
plt.plot(tf.signal.ifft(zeros).numpy().real)
plt.plot(tf.signal.ifft(zeros).numpy().imag)
# plt.plot(tf.signal.fft(tf.signal.ifft(zeros)).numpy().real)
# plt.plot(np.fft.ifft(zeros).real)
# plt.plot(np.fft.ifft(zeros).imag)